In [1]:
# Tlhis Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass#print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Flatten
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image


In [3]:
# Dataset Paths
dataset_path = "/kaggle/input/chest-xray-pneumonia/chest_xray"
train_path = dataset_path + "/train"
test_path = dataset_path + "/test"


In [4]:
# Load the VGG16 model
base_model = VGG16(input_shape=(224, 224, 3), weights='imagenet', include_top=False)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [5]:
# Freeze all the layers in the base model
for layer in base_model.layers:
    layer.trainable = False


In [6]:
x = Flatten()(base_model.output)
x = Dense(256, activation='relu')(x)
x = Dense(2, activation='softmax')(x)  # 2 classes: Normal, Pneumonia


In [7]:
# Define the model
model = Model(inputs=base_model.input, outputs=x)


In [8]:

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load training and test data
training_set = train_datagen.flow_from_directory(train_path, target_size=(224, 224), batch_size=10, class_mode='categorical')
test_set = test_datagen.flow_from_directory(test_path, target_size=(224, 224), batch_size=10, class_mode='categorical')

# Train the model
history = model.fit(training_set, validation_data=test_set, epochs=10)

# Save the model
model.save('chest_xray.h5')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


522/522 ━━━━━━━━━━━━━━━━━━━━ 152s 271ms/step - accuracy: 0.8921 - loss: 0.4660 - val_accuracy: 0.9279 - val_loss: 0.2524
Epoch 2/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 102s 193ms/step - accuracy: 0.9447 - loss: 0.1484 - val_accuracy: 0.9054 - val_loss: 0.3218
Epoch 3/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 97s 184ms/step - accuracy: 0.9621 - loss: 0.1099 - val_accuracy: 0.9022 - val_loss: 0.3363
Epoch 4/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 96s 182ms/step - accuracy: 0.9655 - loss: 0.0912 - val_accuracy: 0.9151 - val_loss: 0.2616
Epoch 5/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 97s 183ms/step - accuracy: 0.9679 - loss: 0.0879 - val_accuracy: 0.9183 - val_loss: 0.2577
Epoch 6/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 97s 183ms/step - accuracy: 0.9730 - loss: 0.0763 - val_accuracy: 0.8798 - val_loss: 0.4127
Epoch 7/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 97s 183ms/step - accuracy: 0.9693 - loss: 0.0777 - val_accuracy: 0.9215 - val_loss: 0.2293
Epoch 8/10
522/522 ━━━━━━━━━━━━━━━━━━━━ 96s 181ms/step - accuracy: 0.9761 - loss: 0.0728 - v

In [11]:
from keras.applications.vgg16 import preprocess_input

FileUpload(value=(), accept='image/*', description='Upload')

In [33]:
def predict(path):
    img=image.load_img(path,target_size=(224,224))
    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    img_data=preprocess_input(x)
    classes=model.predict(img_data)
    result=int(classes[0][0])
    if result==0:
        print("Person is Affected By PNEUMONIA")
    else:
        print("Result is Normal")
    

In [34]:
predict("/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Result is Normal


In [35]:
predict("/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1946_bacteria_4874.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Person is Affected By PNEUMONIA
